### Loading the Data

In [1]:
# importing the necessary Libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')


In [5]:
# Reading the data and changing the data type of the column "DateTime" from string to datetime format
data = pd.read_excel('/Users/pradipwasre/Desktop/Project/data 2/final_customer_data.xlsx')
data['DateTime'] = pd.to_datetime(data['DateTime'])
print(data.shape)
data.head()

(2090, 9)


,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price
0,52243841613,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN
1,52243841613,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN
2,57314161118,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN
3,57314161118,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN
4,57314161118,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   User_id      2090 non-null   int64         
 1   Session_id   2090 non-null   object        
 2   DateTime     2090 non-null   datetime64[ns]
 3   Category     2090 non-null   object        
 4   SubCategory  2090 non-null   object        
 5   Action       2090 non-null   object        
 6   Quantity     201 non-null    float64       
 7   Rate         201 non-null    float64       
 8   Total Price  201 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 147.1+ KB


### RFM Modelling

To do the RFM analysis, we need to create 3 features from the data:
* **Recency**   - Latest date - Last invoice date. (Number of days since the last purchase date)
* **Frequency** - count of invoice numbers. (Total number of transactions made by a unique customer)
* **Monetary**  - Sum of Total sales. (Total value of transacted sales by each customer)

Now, let's create a function which can be used to generate the RFM features.

In [9]:
def RFM_features(df,CustomerID, invoiceDate, transID, sales):
    ''' Create the Recency, Frequency, and Monetary features from the data '''
    latest_data = df[invoiceDate].max() + timedelta(1)

    # RFM feature creation
    RFMScores = df.groupby(CustomerID).agg({invoiceDate: lambda x: (latest_date - x.max()).days,
                                            transID: lambda x:len(x),
                                            sales: lambda x: sum(x)})
    # Converting invoice date to int since this contains number of days
    RFMScores[invoiceDate] = RFMScores[invoiceDate].astype(int)

    # Renaming column names to recency, frequency, and Monetary
    RFMScores.rename(columns = {invoiceDate: 'Racncy',
                                transID: 'Frequency',
                                sales: 'Monetory'}, inplace = TRUE)
    

    return RFMScores.reset_index()